In [1]:
def clip(tmp1):
    import subprocess
    import shlex  # 导入 shlex 模块
    # 使用 shlex.quote 来转义 inp 字符串
    tmp2 = str(tmp1)
    safe_str = shlex.quote(tmp2)
    subprocess.run('echo {} | wclip'.format(safe_str), shell=True)  

def cvin(k):
    clip(In[k])
    
import numpy as np
import matplotlib.pyplot as plt

import time 
#from tqdm import tqdm  # tqdm是显示循环进度条的库
from tqdm.notebook import tqdm #推荐在jupyter中使用自带的进度条
import copy #复制方法

#-------------------------------------------------------------------
np.random.seed(0) #重置种子为0
np.set_printoptions(precision=3, suppress=True, linewidth=100)#格式化输出
#-------------------------------------------------------------------

import rl_utils
import random
import gymnasium as gym
import collections
import torch
import torch.nn.functional as F
import os

import torch

### 基于策略的方法--问题：
基于策略的方法包括策略梯度算法和 Actor-Critic 算法，但是有一个明显的缺点：当策略网络是深度模型时，沿着策略梯度更新参数，很有可能由于步长太长，策略突然显著变差，进而影响训练效果

## TRPO
### 改进：信任区域策略优化(trust region policy optimization，TRPO)
考虑在更新时找到一块 **信任区域（trust region）** ，在这个区域上更新策略时能够得到某种策略性能的安全性保证，这就是 **信任区域策略优化（trust region policy optimization，TRPO）** 算法的主要思想：

> 总的来说，这个推导试图描述新策略相对于旧策略的优势，并使用优势函数$A^\pi_0$来量化这个差异。这是策略梯度方法和TRPO的核心思想
#### 重点：单调性
原因在于，单纯策略梯度下降很有可能是局部的，而使用TRPO能够保证策略学习的性能 **单调性**——即为只要梯度下降，一定是"更好的策略",比策略梯度算法更好的效果

### 1.目标"策略优势"-时序差分残差
> 注意：原公式$s_0$是代表初始状态，但实际上策略价值与初始状态无关，因此才需要改写
#### |通过一系列推导，得出如下公式：
$$
J\left(\theta^{\prime}\right)-J(\theta) = \frac{1}{1-\gamma} \mathbb{E}_{s \sim \nu^{\pi_{\theta^{\prime}}}} \mathbb{E}_{a \sim \pi_{\theta^{\prime}}(\cdot \mid s)}\left[A^{\pi_{\theta}}(s, a)\right]
$$
现在只需要确保这个差值$J\left(\theta^{\prime}\right)-J(\theta) \geqslant 0 $，**就能保证策略性能单调递增**

### 2.替代优化目标
> 直接求解该式是非常困难的，要求解$\pi_\theta^\prime$，而又需要用它来收集样本$v^{\pi_\theta^\prime}$

因此近似操作，忽略两个策略之间的状态访问分布变化，直接采用旧的策略$v^{\pi_\theta}$，进而得到如下替代目标：
$$
L_{\theta}\left(\theta^{\prime}\right)=J(\theta)+\frac{1}{1-\gamma} \mathbb{E}_{s \sim \nu^{\pi_{\theta}}} \mathbb{E}_{a \sim \pi_{\theta^{\prime}}(\cdot \mid s)}\left[A^{\pi_{\theta}}(s, a)\right]
$$
当新旧策略非常接近时，状态访问分布变化很小，这么近似是合理的
#### |用「重要性采样」对动作分布进行处理：
$$
L_{\theta}\left(\theta^{\prime}\right)=J(\theta)+\mathbb{E}_{s \sim \nu^{\pi_{\theta}}} \mathbb{E}_{a \sim \pi_{\theta}(\cdot \mid s)}\left[\frac{\pi_{\theta^{\prime}}(a \mid s)}{\pi_{\theta}(a \mid s)} A^{\pi_{\theta}}(s, a)\right]
$$


### 3."信任区域"
任何近似都有代价的，替代优化目标也不例外，为了保证新旧策略足够接近，TRPO 使用了 **库尔贝克-莱布勒（Kullback-Leibler，KL）** 散度来衡量策略之间的距离，并给出了整体的优化公式：
$$
\begin{aligned} \max _{\theta^{\prime}} & L_{\theta}\left(\theta^{\prime}\right) \\ \text { s.t. } & \mathbb{E}_{s \sim \nu^{\pi_{\theta}}}\left[D_{K L}\left(\pi_{\theta_{k}}(\cdot \mid s), \pi_{\theta^{\prime}}(\cdot \mid s)\right)\right] \leq \delta\end{aligned}
$$
——不等式约束定义了策略空间中的一个KL 球，被称为**信任区域**
> 在这个区域中，可以认为$\pi_\theta$和环境交互的状态分布与上一轮策略最后采样的状态分布一致，进而可以基于一步行动的重要性采样方法使当前学习策略稳定提升

### 4.近似求解
直接求解上式带约束的优化问题比较麻烦，TRPO 在其具体实现中做了一步近似操作来快速求解<br>用$\theta_k$表示这是$k$次迭代之后的策略<br>于是对「替代优化目标」和「信任区域」进行泰勒展开，并取近似：
#### |目标函数 近似：
$$
\mathbb{E}_{s \sim \nu^{\pi_{\theta_{k}}}} \mathbb{E}_{a \sim \pi_{\theta_{k}}(\cdot \mid s)}\left[\frac{\pi_{\theta^{\prime}}(a \mid s)}{\pi_{\theta_{k}}(a \mid s)} A^{\pi_{\theta_{k}}}(s, a)\right] \approx g^{T}\left(\theta^{\prime}-\theta_{k}\right)
$$
其中$g=\nabla_{\theta^{\prime}}L_{\theta}\left(\theta^{\prime}\right)$表示目标函数的梯度
#### |"信任区域"近似：
$$
\mathbb{E}_{s \sim \nu^{\pi_{\theta}}}\left[D_{K L}\left(\pi_{\theta_{k}}(\cdot \mid s), \pi_{\theta^{\prime}}(\cdot \mid s)\right)\right] \approx \frac{1}{2}\left(\theta^{\prime}-\theta_{k}\right)^{T} H\left(\theta^{\prime}-\theta_{k}\right)
$$
其中$H=\mathbf{H}...$为约束条件$\mathbb{E}_{s \sim \nu^{\pi_{\theta}}}\left[D_{K L}\left(\pi_{\theta_{k}}(\cdot \mid s), \pi_{\theta^{\prime}}(\cdot \mid s)\right)\right]$的 **黑塞矩阵**

### 5.最终优化目标：
$$
\theta_{k+1}=\underset{\theta^{\prime}}{\arg \max } g^{T}\left(\theta^{\prime}-\theta_{k}\right) \quad s.t. \quad \frac{1}{2}\left(\theta^{\prime}-\theta_{k}\right)^{T} H\left(\theta^{\prime}-\theta_{k}\right) \leq \delta
$$
<br>用 **卡罗需-库恩-塔克（Karush-Kuhn-Tucker，KKT）** 条件直接导出上述问题的解：
$$
\theta_{k+1}=\theta_{k}+\sqrt{\frac{2 \delta}{g^{T} H^{-1} g}} H^{-1} g
$$
> 注意：这里没有任何有关于梯度下降的东西，这直接就是参数的迭代公式，因为$\theta$就是权重weight！！！

### 6.实际计算：使用「共轭梯度法」绕过$H^{-1}$
用神经网络表示的策略函数的参数数量都是成千上万的，直接计算和存储黑塞矩阵H的逆矩阵，无疑是在犯傻。不过我们可以使用 **共轭梯度法（conjugate gradient method）** 绕过求逆矩阵，详细思路见附录，但是核心思想是直接计算$x=H^{-1}g$，x为 **参数更新方向**：<br>
$$
x = \mathbf{H}^{-1}g
$$
其中x即参数更新方向。
<br>假设满足 KL 距离约束的参数更新时的 **最大步长**为$\beta$，根据KL条件约束，求解方程:$\frac{1}{2}(\beta x)^{T} H(\beta x)=\delta$得到:
$$
\beta=\sqrt{\frac{2 \delta}{x^{T} H x}}
$$
由于$H$为「对称正定矩阵」，因此问题转换为
$$
\theta_{k+1}=\theta_{k}+ \beta x
$$
#### |最终目的：解方程 $Hx=g$解出x
问题转化为解方程 $Hx=g$，因此，只要解出x，或者直接计算$x=H^{-1}g$，就可以根根据该式更新参数。

#### |详细算法流程：
目标是解线性方程组$Hx = g$, 其中$H$是黑塞矩阵，$g$是梯度向量。

1. **初始化**：
   - 选择一个初始解$x_0$, 通常可以为零向量。
   - 计算初始残差$r_0 = g - Hx_0$, 如果 $x_0$是零向量，那么$r_0 = g$.
   - 设置$p_0 = r_0$, 这是我们的初始搜索方向。

2. **迭代计算**：
   为$k = 0, 1, 2, ...$, 执行以下步骤直到$r_k$足够小或达到预设的迭代次数：

   a. 计算步长：  
   $$
   \alpha_k = \frac{r_k^T r_k}{p_k^T H p_k}
   $$

   b. 更新解：  
   $$
   x_{k+1} = x_k + \alpha_k p_k
   $$

   c. 计算新的残差：  
   $$
   r_{k+1} = r_k - \alpha_k H p_k
   $$

   d. 更新方向：  
   $$
   \beta_{k+1} = \frac{r_{k+1}^T r_{k+1}}{r_k^T r_k}
   $$
   $$
   p_{k+1} = r_{k+1} + \beta_{k+1} p_k
   $$

3. **完成**：
   当残差 $r_k$足够小或达到预设的迭代次数时，当前的$x_k$就是我们的解，也就是$x = H^{-1}g$

#### |解释：

共轭梯度法是一个迭代的方法，它试图找到一个方向，使得在这个方向上函数值下降最快。与梯度下降法不同，共轭梯度法每次不仅仅是沿着梯度方向移动，还考虑了前几步的移动方向，以确保每个方向都是“共轭”的。这意味着每次迭代都会选择一个与前几次迭代方向都正交的新方向。

在解$Hx = g$的过程中，我们实际上是在尝试找到使得二次函数$\phi(x) = \frac{1}{2} x^T H x - g^T x$最小的$x$值。每次迭代都会沿着一个新的方向$p_k$移动，直到找到这个最小值。

通过这种方法，共轭梯度法通常可以在比梯度下降法少得多的迭代次数内找到解，特别是当$H$是大的和稀疏的时。

#### |实际解出x算法流程：
- 初始化$ r_{0}=g-H x_{0}$, $p_{0}=r_{0}$, $x_{0}=0$
- **for** $k=0 \rightarrow N$ **do :**
- ----$\alpha_{k}=\frac{r_{k}^{T} r_{k}}{p_{k}^{T} H p_{k}}$
- ----$x_{k+1}=x_{k}+\alpha_{k} p_{k}$
- ----$r_{k+1}=r_{k}-\alpha_{k} H p_{k}$
- ----如果 $r_{k+1}^{T} r_{k+1}$ 非常小, 则退出循环
- ----$\beta_{k}=\frac{r_{k+1}^{T} r_{k+1}}{r_{k}^{T} r_{k}}$
- ----$p_{k+1}=r_{k+1}+\beta_{k} p_{k}$
- **end for**
- 输出 $x_{N+1}$

#### |代码实现
```python
def conjugate_gradient(self, grad, states, old_action_dists):  # 共轭梯度法求解方程
    x = torch.zeros_like(grad)
    r = grad.clone()
    p = grad.clone()
    rdotr = torch.dot(r, r)
    for i in range(10):  # 共轭梯度主循环
        Hp = self.hessian_matrix_vector_product(states, old_action_dists,
                                                p)
        alpha = rdotr / torch.dot(p, Hp)
        x += alpha * p
        r -= alpha * Hp
        new_rdotr = torch.dot(r, r)
        if new_rdotr < 1e-10:
            break
        beta = new_rdotr / rdotr
        p = r + beta * p
        rdotr = new_rdotr
    return x
```

### 7.线性搜索
由于 TRPO 算法用到了泰勒展开的 1 阶和 2 阶近似，这并非精准求解,因此$\theta^\prime$未必比$\theta_k$好，或者未必能满足 KL 散度限制。
<br>TRPO 在每次迭代的最后进行一次 **线性搜索（Line Search）**，以确保找到满足条件，具体就是：
<br>找到一个最小的非负整数i，使得$\theta_{k+1}$仍然满足KL限制，并且确实能够提升$L_{\theta_k}$
$$
\theta_{k+1}=\theta_{k}+\alpha^{i} \sqrt{\frac{2 \delta}{x^{T} H x}} x
$$
其中$\alpha \in (0,1)$是一个超参数

### 8.广义优势估计
我们尚未得知如何估计优势函数A，目前比较常用的一种方法为 **广义优势估计（Generalized Advantage Estimation，GAE）**：<br>
其中 **时序差分误差** ：为$\delta_{t}=r_{t}+\gamma V\left(s_{t+1}\right)-V\left(s_{t}\right)$<br>
- 根据多步时序差分有：
$$
\begin{array}{lc}A_{t}^{(1)}=\delta_{t} & =-V\left(s_{t}\right)+r_{t}+\gamma V\left(s_{t+1}\right) \\ A_{t}^{(2)}=\delta_{t}+\gamma \delta_{t+1} & =-V\left(s_{t}\right)+r_{t}+\gamma r_{t+1}+\gamma^{2} V\left(s_{t+2}\right) \\ A_{t}^{(3)}=\delta_{t}+\gamma \delta_{t+1}+\gamma^{2} \delta_{t+2} & =-V\left(s_{t}\right)+r_{t}+\gamma r_{t+1}+\gamma^{2} r_{t+2}+\gamma^{3} V\left(s_{t+3}\right) \\ \vdots & \vdots \\ A_{t}^{(k)}=\sum_{l=0}^{k-1} \gamma^{l} \delta_{t+l} & =-V\left(s_{t}\right)+r_{t}+\gamma r_{t+1}+\ldots+\gamma^{k-1} r_{t+k-1}+\gamma^{k} V\left(s_{t+k}\right)\end{array}
$$
- GAE的思想非常暴力，就是按照$\gamma^t$来给所有V加权平均：
$$
\begin{aligned} A_{t}^{G A E} & =(1-\lambda)\left(A_{t}^{(1)}+\lambda A_{t}^{(2)}+\lambda^{2} A_{t}^{(3)}+\cdots\right) \\ & =(1-\lambda)\left(\delta_{t}+\lambda\left(\delta_{t}+\gamma \delta_{t+1}\right)+\lambda^{2}\left(\delta_{t}+\gamma \delta_{t+1}+\gamma^{2} \delta_{t+2}\right)+\cdots\right) \\ & =(1-\lambda)\left(\delta\left(1+\lambda+\lambda^{2}+\cdots\right)+\gamma \delta_{t+1}\left(\lambda+\lambda^{2}+\lambda^{3}+\cdots\right)+\gamma^{2} \delta_{t+2}\left(\lambda^{2}+\lambda^{3}+\lambda^{4}+\ldots\right)+\cdots\right) \\ & =(1-\lambda)\left(\delta_{t} \frac{1}{1-\lambda}+\gamma \delta_{t+1} \frac{\lambda}{1-\lambda}+\gamma^{2} \delta_{t+2} \frac{\lambda^{2}}{1-\lambda}+\cdots\right) \\ & =\sum_{l=0}^{\infty}(\gamma \lambda)^{l} \delta_{t+l}\end{aligned}
$$
#### |其中$\lambda \in [0,1]$是GAE额外的超参数：
- $\lambda = 0$,$A_{t}^{G A E}=\delta_{t}=r_{t}+\gamma V\left(s_{t+1}\right)-V\left(s_{t}\right)$即是仅仅只看一步差分得到的优势
- $\lambda = 1$,$A_{t}^{G A E}=\sum_{l=0}^{\infty} \gamma^{l} \delta_{t+l}=\sum_{l=0}^{\infty} \gamma^{l} r_{t+l}-V\left(s_{t}\right)$则是看每一步差分得到优势的完全平均值
#### |GAE核心代码
需要给定$\gamma$,$\lambda$,每个时间步$\delta_{t}$，直接进行优势估计：
```python
def compute_advantage(gamma, lmbda, td_delta):
    td_delta = td_delta.detach().numpy()
    advantage_list = []
    advantage = 0.0
    for delta in td_delta[::-1]:
        advantage = gamma * lmbda * advantage + delta
        advantage_list.append(advantage)
    advantage_list.reverse()
    return torch.tensor(advantage_list, dtype=torch.float)
```

#### |提高稳定性：标准化
在rl_utils.py中，发现了对advantage使用标准化的方法：
标准化是一种常用的数据处理技术，用于将数据调整为零均值和单位标准差。标准化的目的是使得数据的尺度统一，并可能帮助某些算法更快地收敛或提高性能。

这种数学处理方法叫做“标准化” (Standardization) 或者“z-得分标准化” (z-score normalization)。

当我们对数据进行标准化时，我们实际上是在计算数据与其均值的差，并通过标准差来缩放它。这导致了标准化后的数据具有零均值$ \mu = 0$和单位标准差$\sigma = 1$

数学公式如下：

$$
z = \frac{x - \mu}{\sigma}
$$

其中：
- $x$是原始数据点。
- $\mu$是数据的均值。
- $\sigma $是数据的标准差。
- $z$是标准化后的数据点。

这种转换对于许多机器学习和统计方法是有益的，因为它确保了数据在所有特征或维度上的尺度都是相同的。这可以帮助算法更快地收敛，避免某些特征由于其较大的尺度而过度影响算法。
<br>在原始的 `compute_advantage` 函数中，一旦计算了优势 (advantage)，就对其进行了标准化，方法如下：
$$
\text{advantage\_list} = \frac{\text{advantage\_list} - \text{mean}(\text{advantage\_list})}{\text{std}(\text{advantage\_list}) + 1e-5}
$$
这里，$\text{mean}(\text{advantage\_list})$ 是优势的平均值，$\text{std}(\text{advantage\_list})$是优势的标准差。$1e-5$是一个小的常数，用于避免除以零。
<br>通过这种方式，标准化后的优势将有一个均值接近 0 和一个标准差接近 1。标准化优势可以帮助稳定策略的学习，因为它确保了优势的尺度始终相同，不受原始优势尺度的影响:
```python
def compute_advantage(gamma, lmbda, td_delta):
    td_delta = td_delta.detach().numpy()
    advantage_list = []
    advantage = 0.0
    for delta in td_delta[::-1]:  # 逆向折算
        advantage = gamma * lmbda * advantage + delta
        advantage_list.append(advantage)
    advantage_list.reverse()
    advantage_list = torch.tensor(np.array(advantage_list), dtype=torch.float)
    # 对advantage_list进行标准化, 因为优势决定了优化方向
    # 有的优势虽然是正的，但是很小，就应该弱化这种优势，标准化后就会变成负的
    # 当然也可以直接输出advantage_list
    advantage_list = (advantage_list - advantage_list.mean()) / (advantage_list.std() + 1e-5)
    return advantage_list
```

In [ ]:
class PolicyNet(torch.nn.Module):
    def __init__(self, state_dim, hidden_dim, action_dim):
        super(PolicyNet, self).__init__()
        self.fc1 = torch.nn.Linear(state_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, action_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        return F.softmax(self.fc2(x), dim=1)


class ValueNet(torch.nn.Module):
    def __init__(self, state_dim, hidden_dim):
        super(ValueNet, self).__init__()
        self.fc1 = torch.nn.Linear(state_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        return self.fc2(x)


class TRPO:
    """ TRPO算法 """
    def __init__(self, hidden_dim, state_space, action_space, lmbda,
                 kl_constraint, alpha, critic_lr, gamma, device):
        state_dim = state_space.shape[0]
        action_dim = action_space.n
        # 策略网络参数不需要优化器更新
        self.actor = PolicyNet(state_dim, hidden_dim, action_dim).to(device)
        self.critic = ValueNet(state_dim, hidden_dim).to(device)
        self.critic_optimizer = torch.optim.Adam(self.critic.parameters(),
                                                 lr=critic_lr)
        self.gamma = gamma
        self.lmbda = lmbda  # GAE参数
        self.kl_constraint = kl_constraint  # KL距离最大限制
        self.alpha = alpha  # 线性搜索参数
        self.device = device

    def take_action(self, state):
        state = torch.tensor([state], dtype=torch.float).to(self.device)
        probs = self.actor(state)
        action_dist = torch.distributions.Categorical(probs)
        action = action_dist.sample()
        return action.item()

    def hessian_matrix_vector_product(self, states, old_action_dists, vector):
        # 计算黑塞矩阵和一个向量的乘积
        new_action_dists = torch.distributions.Categorical(self.actor(states))
        kl = torch.mean(
            torch.distributions.kl.kl_divergence(old_action_dists,
                                                 new_action_dists))  # 计算平均KL距离
        kl_grad = torch.autograd.grad(kl,
                                      self.actor.parameters(),
                                      create_graph=True)
        kl_grad_vector = torch.cat([grad.view(-1) for grad in kl_grad])
        # KL距离的梯度先和向量进行点积运算
        kl_grad_vector_product = torch.dot(kl_grad_vector, vector)
        grad2 = torch.autograd.grad(kl_grad_vector_product,
                                    self.actor.parameters())
        grad2_vector = torch.cat([grad.view(-1) for grad in grad2])
        return grad2_vector

    def conjugate_gradient(self, grad, states, old_action_dists):  # 共轭梯度法求解方程
        x = torch.zeros_like(grad)
        r = grad.clone()
        p = grad.clone()
        rdotr = torch.dot(r, r)
        for i in range(10):  # 共轭梯度主循环
            Hp = self.hessian_matrix_vector_product(states, old_action_dists,
                                                    p)
            alpha = rdotr / torch.dot(p, Hp)
            x += alpha * p
            r -= alpha * Hp
            new_rdotr = torch.dot(r, r)
            if new_rdotr < 1e-10:
                break
            beta = new_rdotr / rdotr
            p = r + beta * p
            rdotr = new_rdotr
        return x

    def compute_surrogate_obj(self, states, actions, advantage, old_log_probs,
                              actor):  # 计算策略目标
        log_probs = torch.log(actor(states).gather(1, actions))
        ratio = torch.exp(log_probs - old_log_probs)
        return torch.mean(ratio * advantage)

    def line_search(self, states, actions, advantage, old_log_probs,
                    old_action_dists, max_vec):  # 线性搜索
        old_para = torch.nn.utils.convert_parameters.parameters_to_vector(
            self.actor.parameters())
        old_obj = self.compute_surrogate_obj(states, actions, advantage,
                                             old_log_probs, self.actor)
        for i in range(15):  # 线性搜索主循环
            coef = self.alpha**i
            new_para = old_para + coef * max_vec
            new_actor = copy.deepcopy(self.actor)
            torch.nn.utils.convert_parameters.vector_to_parameters(
                new_para, new_actor.parameters())
            new_action_dists = torch.distributions.Categorical(
                new_actor(states))
            kl_div = torch.mean(
                torch.distributions.kl.kl_divergence(old_action_dists,
                                                     new_action_dists))
            new_obj = self.compute_surrogate_obj(states, actions, advantage,
                                                 old_log_probs, new_actor)
            if new_obj > old_obj and kl_div < self.kl_constraint:
                return new_para
        return old_para

    def policy_learn(self, states, actions, old_action_dists, old_log_probs,
                     advantage):  # 更新策略函数
        surrogate_obj = self.compute_surrogate_obj(states, actions, advantage,
                                                   old_log_probs, self.actor)
        grads = torch.autograd.grad(surrogate_obj, self.actor.parameters())
        obj_grad = torch.cat([grad.view(-1) for grad in grads]).detach()
        # 用共轭梯度法计算x = H^(-1)g
        descent_direction = self.conjugate_gradient(obj_grad, states,
                                                    old_action_dists)

        Hd = self.hessian_matrix_vector_product(states, old_action_dists,
                                                descent_direction)
        max_coef = torch.sqrt(2 * self.kl_constraint /
                              (torch.dot(descent_direction, Hd) + 1e-8))
        new_para = self.line_search(states, actions, advantage, old_log_probs,
                                    old_action_dists,
                                    descent_direction * max_coef)  # 线性搜索
        torch.nn.utils.convert_parameters.vector_to_parameters(
            new_para, self.actor.parameters())  # 用线性搜索后的参数更新策略

    def update(self, transition_dict):
        states = torch.tensor(transition_dict['states'],
                              dtype=torch.float).to(self.device)
        actions = torch.tensor(transition_dict['actions']).view(-1, 1).to(
            self.device)
        rewards = torch.tensor(transition_dict['rewards'],
                               dtype=torch.float).view(-1, 1).to(self.device)
        next_states = torch.tensor(transition_dict['next_states'],
                                   dtype=torch.float).to(self.device)
        dones = torch.tensor(transition_dict['dones'],
                             dtype=torch.float).view(-1, 1).to(self.device)
        td_target = rewards + self.gamma * self.critic(next_states) * (1 -
                                                                       dones)
        td_delta = td_target - self.critic(states)
        advantage = compute_advantage(self.gamma, self.lmbda,
                                      td_delta.cpu()).to(self.device)
        old_log_probs = torch.log(self.actor(states).gather(1,
                                                            actions)).detach()
        old_action_dists = torch.distributions.Categorical(
            self.actor(states).detach())
        critic_loss = torch.mean(
            F.mse_loss(self.critic(states), td_target.detach()))
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()  # 更新价值函数
        # 更新策略函数
        self.policy_learn(states, actions, old_action_dists, old_log_probs,
                          advantage)

In [ ]:
%截获机/干扰机
if intc_flag %若在截获状态
    intc_win(2:end,:) = intc_win(1:end-1,:);
    intc_win(1) = freq.*a1;%更新截获频点记录，0表示未截获频点
    intc_judge = (max(intc_win)-min(intc_win) == 0) + ( ...
    
    %判断下一次是否要进入干扰
    if sum(intc_judge)
        intc_flag = 0;%下一脉冲进入干扰
        intf_list = intc_flag .* intc_judge;%生成待干扰频点...
        intf_list = intf_list(intf_list);%生成待干扰频点列表...
        intf_list = kron(ones(1,intf_times),intf_list);%...
        beam_dir_list = find(intc_judge);%生成待干扰雷达列表...
        beam_dir_list = kron(ones(1,intf_times),beam_dir_list)...
    end

elseif ~intc_flag %若在干扰状态
    intf_count = intf_count + 1;
    fj = intf_list(intf_count);%干扰频点
    beam_dir = beam_dir_list(intf_count);%波束指向
    
    %判断干扰是否结束
    if intf_count == intf_times*sum(intc_judge)
        intc_flag = 1;%干扰结束，下一脉冲开始截获
        intf_count = 0;%清空干扰计数
    end
end


## 附录：

### |$J\left(\theta^{\prime}\right)-J(\theta)$详细推导
1. 将$s_0$/$\pi_\theta$期望重新表示为$\pi_\theta^\prime$另一个策略的期望<br>注意这个公式的目的，这实际上意味着只留下了t=0的项
$$
\begin{aligned} J(\theta) & =\mathbb{E}_{s_{0}}\left[V^{\pi_{\theta}}\left(s_{0}\right)\right] \\ & =\mathbb{E}_{\pi_{\theta^{\prime}}}\left[\sum_{t=0}^{\infty} \gamma^{t} V^{\pi_{\theta}}\left(s_{t}\right)-\sum_{t=1}^{\infty} \gamma^{t} V^{\pi_{\theta}}\left(s_{t}\right)\right] \\ & =-\mathbb{E}_{\pi_{\theta^{\prime}}}\left[\sum_{t=0}^{\infty} \gamma^{t}\left(\gamma V^{\pi_{\theta}}\left(s_{t+1}\right)-V^{\pi_{\theta}}\left(s_{t}\right)\right)\right]\end{aligned}
$$
2. 接着就可以用新旧函数作差了<br>首先，给出策略价值差异的直观定义，然后再重写，其中$\pi_\theta^\prime$直接用$G_t$累计回报的定义：
$$
\begin{aligned} J\left(\theta^{\prime}\right)-J(\theta) & =\mathbb{E}_{s_{0}}\left[V^{\pi_{\theta^{\prime}}}\left(s_{0}\right)\right]-\mathbb{E}_{s_{0}}\left[V^{\pi_{\theta}}\left(s_{0}\right)\right] \\ & =\mathbb{E}_{\pi_{\theta^{\prime}}}\left[\sum_{t=0}^{\infty} \gamma^{t} r\left(s_{t}, a_{t}\right)\right]+\mathbb{E}_{\pi_{\theta^{\prime}}}\left[\sum_{t=0}^{\infty} \gamma^{t}\left(\gamma V^{\pi_{\theta}}\left(s_{t+1}\right)-V^{\pi_{\theta}}\left(s_{t}\right)\right)\right] \\ & =\mathbb{E}_{\pi_{\theta^{\prime}}}\left[\sum_{t=0}^{\infty} \gamma^{t}\left[r\left(s_{t}, a_{t}\right)+\gamma V^{\pi_{\theta}}\left(s_{t+1}\right)-V^{\pi_{\theta}}\left(s_{t}\right)\right]\right]\end{aligned}
$$
3. 将时序差分残差定义为优势函数A：
状态访问分布的定义$\nu^{\pi}(s)=(1-\gamma) \sum_{t=0}^{\infty} \gamma^{t} P_{t}^{\pi}(s)$，其中$P_{t}^{\pi}(s)$表示描述了在时刻t下，状态s被访问的概率，而$(1-\gamma)$确保概率之和为1(详细见笔记2-Markov)
$$
\begin{array}{l}=\mathbb{E}_{\pi_{\theta^{\prime}}}\left[\sum_{t=0}^{\infty} \gamma^{t} A^{\pi_{\theta}}\left(s_{t}, a_{t}\right)\right] \\ =\sum_{t=0}^{\infty} \gamma^{t} \mathbb{E}_{s_{t} \sim P_{t}^{\pi_{\theta^{\prime}}}} \mathbb{E}_{a_{t} \sim \pi_{\theta^{\prime}}\left(\cdot \mid s_{t}\right)}\left[A^{\pi_{\theta}}\left(s_{t}, a_{t}\right)\right] \\ =\frac{1}{1-\gamma} \mathbb{E}_{s \sim \nu^{\pi_{\theta^{\prime}}}} \mathbb{E}_{a \sim \pi_{\theta^{\prime}}(\cdot \mid s)}\left[A^{\pi_{\theta}}(s, a)\right]\end{array}
$$

### |重要性采样
$$
\pi(x) f(x)=p(x) \frac{\pi(x)}{p(x)} f(x)
$$
重要性采样的关键不是说原始分布π(x)未知，所以我们要用p(x）来做样本生成。而是因为相同的样本量，用π(x)分布采样得到的结果方差较大，而用p(x)采样的样本得到的结果方差较小。所以大家困惑重要性权重怎么计算，其实这不是个问题，因为我们是知道π(x)的。

$\frac{\pi_{\theta^{\prime}}(a \mid s)}{\pi_{\theta}(a \mid s)}$重要性采样的权重，用于调整了动作a在新策略$\pi_\theta^\prime$和旧策略$\pi_\theta$之间的相对概率，这允许我们使用旧策略的数据来估计新策略的性能或价值

### |KL散度


### |详细过程「 近似求解」
这里使用了泰勒展开的近似
目标函数$ L$是关于$\theta^{\prime}$的。为简化，我们将 $\theta^{\prime}$ 看作一个变量$x$。我们希望在 $\theta_{k}$即$a $点附近展开它。

1. **一阶泰勒展开**:
   对于给定的目标函数 $L$，其一阶泰勒展开是：

$$
L(\theta^{\prime}) \approx L(\theta_{k}) + \nabla_{\theta^{\prime}} L(\theta_{k})^T (\theta^{\prime} - \theta_{k})
$$

其中，$\nabla_{\theta^{\prime}} L(\theta_{k})$是$L $在$\theta_{k}$的梯度。

2. **二阶泰勒展开**:
   对于约束$D_{KL}$，其二阶泰勒展开为：
$$
D_{KL}(\theta^{\prime}) \approx D_{KL}(\theta_{k}) + \nabla_{\theta^{\prime}} D_{KL}(\theta_{k})^T (\theta^{\prime} - \theta_{k}) + \frac{1}{2} (\theta^{\prime} - \theta_{k})^T \nabla^2_{\theta^{\prime}} D_{KL}(\theta_{k}) (\theta^{\prime} - \theta_{k})
$$
其中，$\nabla_{\theta^{\prime}} D_{KL}(\theta_{k})$是$D_{KL}$在$\theta_{k} $的梯度，而$\nabla^2_{\theta^{\prime}} D_{KL}(\theta_{k})$是在$\theta_{k} $的黑塞矩阵。

#### 黑塞矩阵
简而言之，就是用来求二阶偏导的
包含混合偏导数的情况一共有9个，如果用矩阵形式来表示，则可得到：这个矩阵就称为黑塞矩阵（Hessian）
$$
H=\left[\begin{array}{lll}\frac{\partial^{2} f}{\partial x \partial x} & \frac{\partial^{2} f}{\partial x \partial y} & \frac{\partial^{2} f}{\partial x \partial z} \\ \frac{\partial^{2} f}{\partial y \partial x} & \frac{\partial^{2} f}{\partial y \partial y} & \frac{\partial^{2} f}{\partial y \partial z} \\ \frac{\partial^{2} f}{\partial z \partial x} & \frac{\partial^{2} f}{\partial z \partial y} & \frac{\partial^{2} f}{\partial z \partial z}\end{array}\right]
$$

### |卡罗需-库恩-塔克(Karush-Kuhn-Tucker，KKT)条件

### |为何$H$为「对称正定矩阵」？
- 对称性：Hessian矩阵是一个二阶导数矩阵。对于任何充分平滑的函数，其Hessian矩阵都是对称的
- 正定性：对于KL散度而定义的

### |什么是「共轭梯度法」



### 「共轭梯度法」简介
 **共轭梯度法（Conjugate Gradient Method, CGM）**是一种用于求解大规模稀疏线性系统的迭代方法。特别是，它用于求解形式为 \(Ax = b\) 的线性方程，其中 \(A\) 是对称正定矩阵。

CGM的基本思想是：在搜索解的过程中，每次选择一个与前面的所有搜索方向都共轭的搜索方向，这样可以保证在最多 \(n\) 步之内找到确切的解，其中 \(n\) 是矩阵的维度。

### 使用「共轭梯度法」在PyTorch中更新网络
为了使用CGM在PyTorch中，我们首先需要定义一个方法来计算向量与黑塞矩阵的乘积。通常，我们不直接计算黑塞矩阵，而是利用向量-雅可比乘积来计算黑塞矩阵与给定向量的乘积。
以下是使用共轭梯度法在PyTorch中的基本步骤：
1. **定义Hessian-vector积的计算方法**：这可以通过向前向后传递来计算，首先计算损失的梯度，然后使用这些梯度计算向量与Hessian的乘积。
2. **实现共轭梯度法**：使用上面的方法来计算Hessian-vector积，并迭代地找到解。
3. **参数更新**：使用上面的解来更新神经网络的参数。
